In [ ]:
import math
import os
import sys
from itertools import combinations
from pathlib import Path
import pandas as pd

# -----------------------------
# Configuration constants
# -----------------------------
NUM_DEPOTS = 1
NUM_CUSTOMERS = 500
CHUNK_SIZE = 500
SPEED_KMH = 90

FOLDER_FOR_DISTANCES = './distances_to_depot_version_'+ str(NUM_CUSTOMERS) + '_deliveries'
os.makedirs(FOLDER_FOR_DISTANCES, exist_ok=True) # Create folder first

INPUT_XLSX = "initial_data_info.xlsx"
# Prepend folder path to output files
CUSTOMERS_OUT = os.path.join(FOLDER_FOR_DISTANCES, "selected_customers.csv")
DEPOT_OUT = os.path.join(FOLDER_FOR_DISTANCES, "selected_depot.csv")
OUT_CSV = os.path.join(FOLDER_FOR_DISTANCES, "distances_depot_costumers.csv")



# -----------------------------
# Helpers
# -----------------------------
def load_sheet(xls_path: str, sheet_name: str) -> pd.DataFrame:
    xls = pd.ExcelFile(xls_path, engine="openpyxl")
    if sheet_name in xls.sheet_names:
        return pd.read_excel(xls, sheet_name=sheet_name)
    lower_map = {s.lower(): s for s in xls.sheet_names}
    if sheet_name.lower() in lower_map:
        return pd.read_excel(xls, sheet_name=lower_map[sheet_name.lower()])
    raise ValueError(f"Sheet '{sheet_name}' not found.")

def find_col(df: pd.DataFrame, *key_parts: str) -> str:
    for c in df.columns:
        cl = c.lower()
        if all(k in cl for k in key_parts):
            return c
    for key in key_parts:
        for c in df.columns:
            if key in c.lower():
                return c
    raise KeyError(f"Could not find column with keys {key_parts}")

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    dlat, dlon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    a = math.sin(dlat / 2.0)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2.0)**2
    return R * 2.0 * math.asin(math.sqrt(a))

# -----------------------------
# 1) Ensure selections exist
# -----------------------------
if not Path(CUSTOMERS_OUT).exists() or not Path(DEPOT_OUT).exists():
    customers_df = load_sheet(INPUT_XLSX, "customer")
    depots_df = load_sheet(INPUT_XLSX, "depot")

    selected_customers_temp = customers_df.head(NUM_CUSTOMERS).copy()
    selected_depot_temp = depots_df.head(NUM_DEPOTS).copy()

    selected_customers_temp.to_csv(CUSTOMERS_OUT, index=False, encoding="utf-8-sig")
    selected_depot_temp.to_csv(DEPOT_OUT, index=False, encoding="utf-8-sig")

customers_df = pd.read_csv(CUSTOMERS_OUT)
depots_df = pd.read_csv(DEPOT_OUT)
selected_customers = customers_df.head(NUM_CUSTOMERS).copy()
selected_depots = depots_df.head(NUM_DEPOTS).copy()

# -----------------------------
# 2) Identify columns
# -----------------------------
cust_id_col, cust_lat_col, cust_lon_col = find_col(selected_customers, "id"), find_col(selected_customers, "lat"), find_col(selected_customers, "lon")
depot_id_col, depot_lat_col, depot_lon_col = find_col(selected_depots, "id"), find_col(selected_depots, "lat"), find_col(selected_depots, "lon")

# -----------------------------
# 3) Compute distances
# -----------------------------
rows_buffer, total_rows, chunk_count = [], 0, 0

def flush_rows(buffer, base_path):
    global chunk_count
    if not buffer: return
    chunk_count += 1
    
    path_obj = Path(base_path)
    # Ensure chunk files are saved inside the folder by using path_obj.parent
    chunk_file = path_obj.parent / f"{path_obj.stem}_{chunk_count}{path_obj.suffix}"
    
    df_chunk = pd.DataFrame(buffer, columns=["id_node1", "id_node2", "distance_km"])
    df_chunk["time_h"] = df_chunk["distance_km"] / SPEED_KMH
    
    df_chunk.to_csv(chunk_file, index=False, encoding="utf-8-sig")
    print(f"  Saved {len(buffer)} rows -> {chunk_file}")
    buffer.clear()

nodes = []
if not selected_depots.empty:
    d = selected_depots.iloc[0]
    nodes.append({"id": str(d[depot_id_col]), "lat": float(d[depot_lat_col]), "lon": float(d[depot_lon_col])})

for _, c in selected_customers.iterrows():
    nodes.append({"id": str(c[cust_id_col]), "lat": float(c[cust_lat_col]), "lon": float(c[cust_lon_col])})

print(f"Computing distances for {len(nodes)} nodes...")
pair_count = 0
for (n1, n2) in combinations(nodes, 2):
    rows_buffer.append({"id_node1": n1["id"], "id_node2": n2["id"], "distance_km": haversine_km(n1["lat"], n1["lon"], n2["lat"], n2["lon"])})
    pair_count += 1
    if len(rows_buffer) >= CHUNK_SIZE:
        total_rows += len(rows_buffer)
        flush_rows(rows_buffer, OUT_CSV)

if rows_buffer:
    total_rows += len(rows_buffer)
    flush_rows(rows_buffer, OUT_CSV)

print(f"Written {total_rows:,} total rows across {chunk_count} files in {FOLDER_FOR_DISTANCES}")
